In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px


class LandslideRiskDashboard:
    """
    Python implementation of the Landslide Risk Assessment Dashboard
    Translates the React/TypeScript functionality into Python with Dash
    """
    
    def __init__(self):
        # Initialize default values for dashboard parameters
        self.confidence_level = 95
        self.region_selected = 'Region A'
        self.time_horizon = 10
        self.mitigation_budget = 5000000
        
        # Define data for different regions (same as in the TypeScript version)
        self.region_data = {
            'Region A': {
                'populationDensity': 1200,
                'infrastructureValue': 45000000,
                'annualIncome': 38000,
                'landValue': 2500,
                'historicalEvents': 12,
                'slope': 32,
                'vegetation': 'Low',
                'rainfall': 'High'
            },
            'Region B': {
                'populationDensity': 850,
                'infrastructureValue': 32000000,
                'annualIncome': 42000,
                'landValue': 3200,
                'historicalEvents': 8,
                'slope': 28,
                'vegetation': 'Medium',
                'rainfall': 'Medium'
            },
            'Region C': {
                'populationDensity': 1800,
                'infrastructureValue': 67000000,
                'annualIncome': 35000,
                'landValue': 4100,
                'historicalEvents': 17,
                'slope': 38,
                'vegetation': 'Very Low',
                'rainfall': 'Very High'
            }
        }
        
        # Colors for visualization (same as in TypeScript)
        self.colors = ['#0088FE', '#00C49F', '#FFBB28', '#FF8042']
        
        # Initialize the Dash app
        self.app = dash.Dash(__name__)
        self.setup_layout()
        self.setup_callbacks()
    
    def calculate_var(self, confidence, region, horizon, budget):
        """
        Calculate Value at Risk based on inputs.
        Direct translation of the calculateVaR function from TypeScript.
        """
        region_info = self.region_data[region]
        
        # Risk factors based on region characteristics
        population_risk = region_info['populationDensity'] / 1000
        slope_risk = region_info['slope'] / 40
        
        # Mapping text values to numeric factors
        vegetation_mapping = {'Very Low': 0.9, 'Low': 0.7, 'Medium': 0.5, 'High': 0.3, 'Very High': 0.1}
        rainfall_mapping = {'Very Low': 0.1, 'Low': 0.3, 'Medium': 0.5, 'High': 0.7, 'Very High': 0.9}
        
        vegetation_factor = vegetation_mapping[region_info['vegetation']]
        rainfall_factor = rainfall_mapping[region_info['rainfall']]
        historical_factor = min(1, region_info['historicalEvents'] / 20)
        
        # Combined risk score (0-1)
        base_risk_score = (population_risk * 0.2 + slope_risk * 0.25 + 
                          vegetation_factor * 0.2 + rainfall_factor * 0.25 + 
                          historical_factor * 0.1)
        
        # Adjust risk based on confidence level
        confidence_adjustment = (confidence - 50) / 50
        adjusted_risk = base_risk_score * (1 + confidence_adjustment * 0.5)
        
        # Scale by time horizon (risk increases with longer time periods)
        time_adjusted_risk = adjusted_risk * pow(horizon, 0.7)
        
        # Impact calculations
        casualties = round(region_info['populationDensity'] * time_adjusted_risk * 0.01 * horizon / 10)
        infrastructure_loss = region_info['infrastructureValue'] * time_adjusted_risk * 0.2
        land_area_lost = 1000 * time_adjusted_risk  # in hectares
        land_value_loss = land_area_lost * region_info['landValue']
        income_loss = region_info['annualIncome'] * casualties * 5  # 5 years of income loss per casualty
        displacement_count = round(region_info['populationDensity'] * time_adjusted_risk * 0.1)
        displacement_cost = displacement_count * 25000  # Cost per displaced person
        
        total_economic_impact = infrastructure_loss + land_value_loss + income_loss + displacement_cost
        
        # Calculate how mitigation budget would reduce impacts
        mitigation_effectiveness = min(0.8, budget / (total_economic_impact * 0.5))
        mitigated_total_impact = total_economic_impact * (1 - mitigation_effectiveness)
        mitigated_casualties = casualties * (1 - mitigation_effectiveness * 0.9)  # Higher effectiveness for human safety
        
        # Return dictionary with all calculated values
        return {
            'probability': min(0.99, time_adjusted_risk),
            'casualties': round(casualties),
            'mitigatedCasualties': round(mitigated_casualties),
            'infrastructureLoss': infrastructure_loss,
            'landAreaLost': round(land_area_lost, 1),
            'landValueLoss': land_value_loss,
            'incomeLoss': income_loss,
            'displacementCount': displacement_count,
            'displacementCost': displacement_cost,
            'totalEconomicImpact': total_economic_impact,
            'mitigatedEconomicImpact': mitigated_total_impact,
            'returnPeriod': round(1 / min(0.99, time_adjusted_risk)),
            'costBreakdown': [
                {'name': 'Infrastructure', 'value': infrastructure_loss},
                {'name': 'Land Value', 'value': land_value_loss},
                {'name': 'Income Loss', 'value': income_loss},
                {'name': 'Displacement', 'value': displacement_cost}
            ],
            'mitigationROI': (total_economic_impact - mitigated_total_impact) / budget
        }
    
    def generate_var_curve_data(self):
        """Generate data for the VaR curve (how VaR changes with confidence level)"""
        var_curve_data = []
        for conf_level in range(50, 100, 5):
            result = self.calculate_var(conf_level, self.region_selected, self.time_horizon, self.mitigation_budget)
            var_curve_data.append({
                'confidenceLevel': conf_level,
                'economicImpact': result['totalEconomicImpact'] / 1000000,
                'casualties': result['casualties']
            })
        return var_curve_data
    
    def generate_mitigation_data(self):
        """Generate data to show effects of different mitigation budgets"""
        mitigation_data = []
        for budget in range(1, 10000001, 1000000):
            result = self.calculate_var(self.confidence_level, self.region_selected, self.time_horizon, budget)
            mitigation_data.append({
                'budget': budget / 1000000,
                'impact': result['mitigatedEconomicImpact'] / 1000000,
                'casualties': result['mitigatedCasualties'],
                'roi': result['mitigationROI']
            })
        return mitigation_data
    
    def format_currency(self, value):
        """Format value as currency"""
        return f"${value:,.0f}"
    
    def create_cost_breakdown_figure(self, var_results):
        """Create a pie chart for the cost breakdown"""
        labels = [item['name'] for item in var_results['costBreakdown']]
        values = [item['value'] for item in var_results['costBreakdown']]
        
        fig = go.Figure(data=[go.Pie(
            labels=labels,
            values=values,
            hole=0.4,
            marker=dict(colors=self.colors)
        )])
        
        fig.update_layout(
            title='Cost Breakdown',
            margin=dict(t=30, b=0, l=0, r=0)
        )
        
        return fig
    
    def create_var_curve_figure(self, var_curve_data):
        """Create a line chart for the VaR curve"""
        df = pd.DataFrame(var_curve_data)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df['confidenceLevel'],
            y=df['economicImpact'],
            mode='lines+markers',
            name='Economic Impact ($M)',
            line=dict(color='#0088FE', width=2)
        ))
        
        fig.update_layout(
            title='Value at Risk by Confidence Level',
            xaxis_title='Confidence Level (%)',
            yaxis_title='Economic Impact (Millions $)',
            margin=dict(t=30, b=30, l=30, r=10)
        )
        
        return fig
    
    def create_mitigation_figure(self, mitigation_data):
        """Create a line chart for mitigation effectiveness"""
        df = pd.DataFrame(mitigation_data)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df['budget'],
            y=df['impact'],
            mode='lines',
            name='Mitigated Impact ($M)',
            line=dict(color='#00C49F', width=2)
        ))
        
        # Add second y-axis for ROI
        fig.add_trace(go.Scatter(
            x=df['budget'],
            y=df['roi'],
            mode='lines',
            name='ROI',
            line=dict(color='#FF8042', width=2, dash='dash'),
            yaxis='y2'
        ))
        
        fig.update_layout(
            title='Mitigation Effectiveness',
            xaxis_title='Mitigation Budget (Millions $)',
            yaxis_title='Mitigated Impact (Millions $)',
            yaxis2=dict(
                title='Return on Investment (ROI)',
                overlaying='y',
                side='right'
            ),
            margin=dict(t=30, b=30, l=30, r=40),
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
        )
        
        return fig
    
    def setup_layout(self):
        """Set up the Dash layout (UI) for the dashboard"""
        self.app.layout = html.Div([
            html.H1("Landslide Risk Assessment Dashboard", className="dashboard-title"),
            
            # Controls section
            html.Div([
                html.Div([
                    html.Label(f"Risk Confidence Level: {self.confidence_level}%"),
                    dcc.Slider(
                        id='confidence-slider',
                        min=50,
                        max=99,
                        value=self.confidence_level,
                        marks={i: f"{i}" for i in range(50, 100, 10)},
                    ),
                    html.P("Higher values represent more conservative risk estimates", className="hint-text")
                ], className="control-box"),
                
                html.Div([
                    html.Label("Region"),
                    dcc.Dropdown(
                        id='region-dropdown',
                        options=[{'label': region, 'value': region} for region in self.region_data.keys()],
                        value=self.region_selected
                    )
                ], className="control-box"),
                
                html.Div([
                    html.Label(f"Time Horizon: {self.time_horizon} Years"),
                    dcc.Slider(
                        id='time-horizon-slider',
                        min=1,
                        max=50,
                        value=self.time_horizon,
                        marks={i: f"{i}" for i in range(0, 51, 10)},
                    )
                ], className="control-box"),
                
                html.Div([
                    html.Label("Mitigation Budget"),
                    dcc.Slider(
                        id='budget-slider',
                        min=0,
                        max=10000000,
                        step=500000,
                        value=self.mitigation_budget,
                        marks={i: f"${i/1000000}M" for i in range(0, 10000001, 2000000)},
                    ),
                    html.P(id='budget-display', className="value-text")
                ], className="control-box"),
            ], className="controls-container"),
            
            # Key metrics section - will be populated by callback
            html.Div(id='key-metrics', className="metrics-container"),
            
            # Charts section
            html.Div([
                html.Div([
                    dcc.Graph(id='cost-breakdown-chart')
                ], className="chart-box"),
                
                html.Div([
                    dcc.Graph(id='var-curve-chart')
                ], className="chart-box"),
                
                html.Div([
                    dcc.Graph(id='mitigation-chart')
                ], className="chart-box"),
            ], className="charts-container"),
            
            # Explanation section
            html.Div([
                html.H3("About This Dashboard", className="section-title"),
                html.P("""
                    This dashboard applies Value at Risk (VaR) methodology to landslide risk assessment. 
                    By adjusting the confidence level, you're specifying how conservative your risk estimate should be. 
                    For example, a 95% confidence level means there's a 95% probability that losses will not exceed the calculated VaR amount.
                """, className="explanation-text"),
                
                html.Div([
                    html.Div([
                        html.H4("Key Metrics Explained", className="subsection-title"),
                        html.Ul([
                            html.Li(("Value at Risk (VaR): ") + "The maximum economic loss expected with the specified confidence level"),
                            html.Li(("Return Period: ") + "How frequently a landslide event of this magnitude is expected to occur"),
                            html.Li(("Mitigation ROI: ") + "Return on investment for risk reduction measures"),
                        ], className="explanation-list")
                    ], className="explanation-column"),
                    
                    html.Div([
                        html.H4("Decision Support", className="subsection-title"),
                        html.Ul([
                            html.Li("Compare different regions to prioritize interventions"),
                            html.Li("Adjust the mitigation budget to find optimal spending levels"),
                            html.Li("Test different time horizons for short and long-term planning"),
                            html.Li("Analyze cost breakdowns to target specific vulnerability factors"),
                        ], className="explanation-list")
                    ], className="explanation-column"),
                ], className="explanation-columns"),
            ], className="explanation-container"),
            
        ], className="dashboard-container")
    
    def setup_callbacks(self):
        """Set up the interactive callbacks for the dashboard"""
        
        # Update budget display
        @self.app.callback(
            Output('budget-display', 'children'),
            [Input('budget-slider', 'value')]
        )
        def update_budget_display(value):
            return self.format_currency(value)
        
        # Update key metrics based on input changes
        @self.app.callback(
            Output('key-metrics', 'children'),
            [
                Input('confidence-slider', 'value'),
                Input('region-dropdown', 'value'),
                Input('time-horizon-slider', 'value'),
                Input('budget-slider', 'value')
            ]
        )
        def update_metrics(confidence, region, horizon, budget):
            # Update instance variables
            self.confidence_level = confidence
            self.region_selected = region
            self.time_horizon = horizon
            self.mitigation_budget = budget
            
            # Calculate VaR results
            var_results = self.calculate_var(confidence, region, horizon, budget)
            
            # Return the key metrics HTML
            return [
                html.Div([
                    html.H3("Value at Risk (VaR)", className="metric-title"),
                    html.P(self.format_currency(var_results['totalEconomicImpact']), className="metric-value"),
                    html.P(f"at {confidence}% confidence over {horizon} years", className="metric-subtitle"),
                    html.P(f"Probability: {(var_results['probability'] * 100):.1f}% | Return period: 1 in {var_results['returnPeriod']} years", className="metric-detail")
                ], className="metric-box"),
                
                html.Div([
                    html.H3("Estimated Casualties", className="metric-title"),
                    html.P(f"{var_results['casualties']}", className="metric-value"),
                    html.P(f"With mitigation: {var_results['mitigatedCasualties']}", className="metric-subtitle"),
                    html.P(f"Displaced persons: {var_results['displacementCount']:,}", className="metric-detail")
                ], className="metric-box"),
                
                html.Div([
                    html.H3("Land Impact", className="metric-title"),
                    html.P(f"{var_results['landAreaLost']} ha", className="metric-value"),
                    html.P(f"Value: {self.format_currency(var_results['landValueLoss'])}", className="metric-subtitle"),
                ], className="metric-box"),
                
                html.Div([
                    html.H3("Mitigation ROI", className="metric-title"),
                    html.P(f"{var_results['mitigationROI']:.2f}x", className="metric-value"),
                    html.P(f"Risk reduced by {(var_results['mitigationROI'] * 100):.0f}%", className="metric-subtitle"),
                ], className="metric-box"),
            ]
        
        # Update charts based on input changes
        @self.app.callback(
            [
                Output('cost-breakdown-chart', 'figure'),
                Output('var-curve-chart', 'figure'),
                Output('mitigation-chart', 'figure')
            ],
            [
                Input('confidence-slider', 'value'),
                Input('region-dropdown', 'value'),
                Input('time-horizon-slider', 'value'),
                Input('budget-slider', 'value')
            ]
        )
        def update_charts(confidence, region, horizon, budget):
            # Update instance variables
            self.confidence_level = confidence
            self.region_selected = region
            self.time_horizon = horizon
            self.mitigation_budget = budget
            
            # Calculate VaR results
            var_results = self.calculate_var(confidence, region, horizon, budget)
            
            # Generate data for charts
            var_curve_data = self.generate_var_curve_data()
            mitigation_data = self.generate_mitigation_data()
            
            # Create figures
            cost_breakdown_fig = self.create_cost_breakdown_figure(var_results)
            var_curve_fig = self.create_var_curve_figure(var_curve_data)
            mitigation_fig = self.create_mitigation_figure(mitigation_data)
            
            return cost_breakdown_fig, var_curve_fig, mitigation_fig
    
    def run_server(self, debug=True, port=8050):
        """Run the Dash server"""
        self.app.run_server(debug=debug, port=port)

In [5]:
# Example usage
#if __name__ == '__main__':
dashboard = LandslideRiskDashboard()
dashboard.run_server()

In [8]:

# Optional: Function to generate a static report using matplotlib
def generate_landslide_risk_report(region='Region A', confidence_level=95, time_horizon=10, mitigation_budget=5000000):
    """
    Generate a static report of landslide risk assessment
    This can be used without running the full Dash app
    """
    dashboard = LandslideRiskDashboard()
    dashboard.confidence_level = confidence_level
    dashboard.region_selected = region
    dashboard.time_horizon = time_horizon
    dashboard.mitigation_budget = mitigation_budget
    
    # Calculate VaR results
    var_results = dashboard.calculate_var(confidence_level, region, time_horizon, mitigation_budget)
    
    # Set up matplotlib figure
    #plt.style.use('fivethirtyeight')
    plt.style.use('ggplot')
    fig = plt.figure(figsize=(15, 12))
    
    # Create grid for plots
    gs = fig.add_gridspec(3, 3)
    
    # Title
    fig.suptitle(f'Landslide Risk Assessment Report: {region}', fontsize=16)
    
    # Key metrics
    ax_metrics = fig.add_subplot(gs[0, :])
    ax_metrics.axis('off')
    metrics_text = f"""
    Confidence Level: {confidence_level}%   |   Time Horizon: {time_horizon} years   |   Mitigation Budget: ${mitigation_budget:,}
    
    Value at Risk (VaR): ${var_results['totalEconomicImpact']:,.0f}
    Probability: {var_results['probability']*100:.1f}%   |   Return Period: 1 in {var_results['returnPeriod']} years
    
    Estimated Casualties: {var_results['casualties']}   |   With Mitigation: {var_results['mitigatedCasualties']}
    Displaced Persons: {var_results['displacementCount']:,}
    
    Land Area Impact: {var_results['landAreaLost']} ha   |   Value: ${var_results['landValueLoss']:,.0f}
    
    Mitigation ROI: {var_results['mitigationROI']:.2f}x   |   Risk Reduction: {var_results['mitigationROI']*100:.0f}%
    """
    ax_metrics.text(0.05, 0.5, metrics_text, fontsize=12, va='center', ha='left')
    
    # Cost breakdown pie chart
    ax_pie = fig.add_subplot(gs[1, 0])
    labels = [item['name'] for item in var_results['costBreakdown']]
    values = [item['value'] for item in var_results['costBreakdown']]
    ax_pie.pie(values, labels=labels, autopct='%1.1f%%', startangle=90, colors=dashboard.colors)
    ax_pie.set_title('Cost Breakdown')
    
    # VaR curve
    ax_var = fig.add_subplot(gs[1, 1:])
    var_curve_data = dashboard.generate_var_curve_data()
    var_df = pd.DataFrame(var_curve_data)
    ax_var.plot(var_df['confidenceLevel'], var_df['economicImpact'], 'o-', color='#0088FE', linewidth=2)
    ax_var.set_title('Value at Risk by Confidence Level')
    ax_var.set_xlabel('Confidence Level (%)')
    ax_var.set_ylabel('Economic Impact (Millions $)')
    ax_var.grid(True, linestyle='--', alpha=0.7)
    
    # Mitigation effectiveness
    ax_mit = fig.add_subplot(gs[2, :])
    mitigation_data = dashboard.generate_mitigation_data()
    mit_df = pd.DataFrame(mitigation_data)
    
    ax_mit.plot(mit_df['budget'], mit_df['impact'], '-', color='#00C49F', linewidth=2, label='Mitigated Impact')
    ax_mit.set_xlabel('Mitigation Budget (Millions $)')
    ax_mit.set_ylabel('Mitigated Impact (Millions $)')
    
    ax_roi = ax_mit.twinx()
    ax_roi.plot(mit_df['budget'], mit_df['roi'], '--', color='#FF8042', linewidth=2, label='ROI')
    ax_roi.set_ylabel('Return on Investment (ROI)')
    
    lines1, labels1 = ax_mit.get_legend_handles_labels()
    lines2, labels2 = ax_roi.get_legend_handles_labels()
    ax_mit.legend(lines1 + lines2, labels1 + labels2, loc='upper center')
    ax_mit.set_title('Mitigation Effectiveness')
    ax_mit.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    return fig


# Example of generating a static report
fig = generate_landslide_risk_report('Region C', confidence_level=90)
plt.savefig('landslide_risk_report.png', dpi=300)
plt.close()
